# [Optimised Modes](https://openpyxl.readthedocs.io/en/stable/optimized.html)

## Create Target Data

In [1]:
"""
# Simple Usage of openpyxl
"""
import openpyxl as xl
from distutils.dir_util import copy_tree
import pprint as pp

copy_tree('archive/', 'data/')

['data/logo.png']

In [2]:
WB_1_FILE = "data/large_file.xlsx"
WB_2_FILE = "data/new_big_file.xlsx"
WB_3_FILE = "data/write_only_file.xlsx"
WS_1_NAME = "big_data"
WS_2_NAME = "Pi"
WS_3_NAME = "Data"

In [7]:
def write_workbook():
    print(80 * '-')
    print("Write Workbook")
    print(40 * '-')

    wb = xl.Workbook()

    dest_filename = WB_1_FILE

    ws1 = wb.active
    ws1.title = WS_1_NAME

    for row in range(1, 40):
        ws1.append(range(10))

    ws2 = wb.create_sheet(title=WS_2_NAME)
    ws2['A3'] = 3.14

    ws3 = wb.create_sheet(title=WS_3_NAME)
    for row in range(1, 10):
        for col in range(1, 10):
            key = "{0}".format(str(xl.utils.get_column_letter(col)) + str(row))
            _ = ws3.cell(column=col, row=row, value=key)
            print(ws3[key].value)

    wb.save(filename=dest_filename)

In [8]:
write_workbook()

--------------------------------------------------------------------------------
Write Workbook
----------------------------------------
A1
B1
C1
D1
E1
F1
G1
H1
I1
A2
B2
C2
D2
E2
F2
G2
H2
I2
A3
B3
C3
D3
E3
F3
G3
H3
I3
A4
B4
C4
D4
E4
F4
G4
H4
I4
A5
B5
C5
D5
E5
F5
G5
H5
I5
A6
B6
C6
D6
E6
F6
G6
H6
I6
A7
B7
C7
D7
E7
F7
G7
H7
I7
A8
B8
C8
D8
E8
F8
G8
H8
I8
A9
B9
C9
D9
E9
F9
G9
H9
I9


## Read-only mode

Sometimes, you will need to open or write **extremely large XLSX files**, and the common routines in openpyxl won’t be able to handle that load. Fortunately, there are two modes that enable you to read and write unlimited amounts of data with (near) constant memory consumption.

Introducing `openpyxl.worksheet._read_only.ReadOnlyWorksheet:`

In [ ]:
wb = xl.load_workbook(filename=WB_1_FILE, read_only=True)
ws = wb[WS_3_NAME]

for row in ws.rows:
    for cell in row:
        print(cell.value)

# Close the workbook after reading
wb.close()

>Warning
>
>* `openpyxl.worksheet._read_only.ReadOnlyWorksheet` is read-only
>* Unlike a normal workbook, a read-only workbook will use lazy loading. The workbook must be explicitly closed with the `close()` method.

Cells returned are not regular `openpyxl.cell.cell.Cell` but `openpyxl.cell._read_only.ReadOnlyCell`.

### Worksheet dimensions

Read-only mode relies on applications and libraries that created the file providing correct information about the worksheets, specifically the used part of it, known as the dimensions. Some applications set this incorrectly. You can check the apparent dimensions of a worksheet using `ws.calculate_dimension()`. If this returns a range that you know is incorrect, say `A1:A1` then simply resetting the `max_row` and `max_column` attributes should allow you to work with the file:

In [ ]:
ws.reset_dimensions()

## Write-only mode
Here again, the regular `openpyxl.worksheet.worksheet.Worksheet` has been replaced by a faster alternative, the `openpyxl.worksheet._write_only.WriteOnlyWorksheet`. When you want to dump large amounts of data make sure you have lxml installed.

In [ ]:
wb = xl.Workbook(write_only=True)
ws = wb.create_sheet()

# now we'll fill it with 100 rows x 200 columns

for irow in range(100):
    ws.append(['%d' % i for i in range(200)])
# save the file
wb.save(WB_2_FILE) # doctest: +SKIP

If you want to have cells with styles or comments then use a `openpyxl.cell.WriteOnlyCell()`

In [ ]:
wb = xl.Workbook(write_only = True)
ws = wb.create_sheet()
#from openpyxl.cell import WriteOnlyCell
#from openpyxl.comments import Comment
#from openpyxl.styles import Font
cell = xl.cell.WriteOnlyCell(ws, value="hello world")
cell.font = xl.styles.Font(name='Courier', size=36)
cell.comment = xl.comments.Comment(text="A comment", author="Author's Name")
ws.append([cell, 3.14, None])
wb.save(WB_3_FILE)

This will create a write-only workbook with a single sheet, and append a row of 3 cells: one text cell with a custom font and a comment, a floating-point number, and an empty cell (which will be discarded anyway).

>Warning
>
>* Unlike a normal workbook, a newly-created write-only workbook does not contain any worksheets; a worksheet must be specifically created with the create_sheet() method.
>* In a write-only workbook, rows can only be added with append(). It is not possible to write (or read) cells at arbitrary locations with cell() or iter_rows().
>* It is able to export unlimited amount of data (even more than Excel can handle actually), while keeping memory usage under 10Mb.
>* A write-only workbook can only be saved once. After that, every attempt to save the workbook or append() to an existing worksheet will raise an openpyxl.utils.exceptions.WorkbookAlreadySaved exception.
>* Everything that appears in the file before the actual cell data must be created before cells are added because it must written to the file before then. For example, freeze_panes should be set before cells are added.

---
*EOF*